<h2><b>Langchain : Agent</b></h2>

In [3]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

api_wrapper = WikipediaAPIWrapper(top_k_results=1 , doc_content_chars_max=200)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper)

wiki.name

'wikipedia'

In [4]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_google_genai.embeddings import GoogleGenerativeAIEmbeddings
from dotenv import load_dotenv
from langchain_text_splitters import RecursiveCharacterTextSplitter

load_dotenv()

loader = WebBaseLoader("https://docs.smith.langchain.com/")
docs = loader.load()

documents = RecursiveCharacterTextSplitter(chunk_size=1000 , chunk_overlap=200).split_documents(docs)

vectordb = FAISS.from_documents(documents,GoogleGenerativeAIEmbeddings(model="models/embedding-001"))
retriever = vectordb.as_retriever()

retriever

USER_AGENT environment variable not set, consider setting it to identify your requests.


VectorStoreRetriever(tags=['FAISS', 'GoogleGenerativeAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000027B7ABF9C70>, search_kwargs={})

In [5]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(
    retriever,
    "Langsmith_Search",
    "Search for the information about LangSmith, For any Question about Langsmith , you must have be using this tool"
)

retriever_tool.name

'Langsmith_Search'

In [6]:
#Arxiv Tool- Website where all the Research papers Uploaded
from langchain_community.tools import ArxivQueryRun
from langchain_community.utilities import ArxivAPIWrapper

arxiv_wrapper = ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=200)
arxiv = ArxivQueryRun(arxiv_wrapper = arxiv_wrapper)
arxiv.name

'arxiv'

In [7]:
tools=[wiki,arxiv,retriever_tool]
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'd:\\New Folder\\Main\\Coding\\Python\\Data Science\\Projects\\Practice\\GenAI\\langchain Campusx\\langcamp\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=3, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=4000)),
 Tool(name='Langsmith_Search', description='Search for the information about LangSmith, For any Question about Langsmith , you must have be using this tool', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x0000027B69FE0180>, retriever=Vec

In [8]:
from langchain_google_genai.chat_models import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro-latest")

# from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
# from dotenv import load_dotenv

# load_dotenv()

# model = HuggingFaceEndpoint(
#     repo_id="google/flan-t5-large",  # Safe model
#     task="text2text-generation"
# )
# llm = ChatHuggingFace(llm=model)


In [9]:
from langchain import hub

prompt = hub.pull("hwchase17/structured-chat-agent")
prompt.messages

d:\New Folder\Main\Coding\Python\Data Science\Projects\Practice\GenAI\langchain Campusx\langcamp\Lib\site-packages\langsmith\client.py:253: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['tool_names', 'tools'], input_types={}, partial_variables={}, template='Respond to the human as helpfully and accurately as possible. You have access to the following tools:\n\n{tools}\n\nUse a json blob to specify a tool by providing an action key (tool name) and an action_input key (tool input).\n\nValid "action" values: "Final Answer" or {tool_names}\n\nProvide only ONE action per $JSON_BLOB, as shown:\n\n```\n{{\n  "action": $TOOL_NAME,\n  "action_input": $INPUT\n}}\n```\n\nFollow this format:\n\nQuestion: input question to answer\nThought: consider previous and subsequent steps\nAction:\n```\n$JSON_BLOB\n```\nObservation: action result\n... (repeat Thought/Action/Observation N times)\nThought: I know what to respond\nAction:\n```\n{{\n  "action": "Final Answer",\n  "action_input": "Final response to human"\n}}\n\nBegin! Reminder to ALWAYS respond with a valid json blob of a single action. Use tools if necessary. R

In [10]:
# Agents
from langchain.agents import create_structured_chat_agent

agent =create_structured_chat_agent(llm,tools,prompt)
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_log_to_str(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMes

In [11]:
#Agent Executer
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent,tools=tools,verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_log_to_str(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk'

In [12]:
# agent_executor.invoke({"input":"Tell me about Langsmith"})

In [13]:
# agent_executor.invoke({"input":"Tell me about Attention all you need"})

In [14]:
# agent_executor.invoke({"input":"Tell me about the History of India"})

In [16]:
agent_executor.invoke({"input":"Tell me about Machine learning"})



> Entering new AgentExecutor chain...
Thought: I can use wikipedia to get information about Machine Learning.
Action:
```json
{
  "action": "wikipedia",
  "action_input": "Machine learning"
}
```Page: Machine learning
Summary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and genAction:
```json
{
  "action": "Final Answer",
  "action_input": "Machine learning (ML) is a field of study in artificial intelligence that allows computers to learn from data without explicit programming. It involves developing algorithms and statistical models that enable computer systems to identify patterns, make predictions, and improve their performance over time based on the data they are exposed to.  Essentially, machine learning algorithms learn by identifying patterns in data and then using these patterns to make informed decisions and predictions on new data."
}
```

> Finished chain.


{'input': 'Tell me about Machine learning',
 'output': 'Machine learning (ML) is a field of study in artificial intelligence that allows computers to learn from data without explicit programming. It involves developing algorithms and statistical models that enable computer systems to identify patterns, make predictions, and improve their performance over time based on the data they are exposed to.  Essentially, machine learning algorithms learn by identifying patterns in data and then using these patterns to make informed decisions and predictions on new data.'}